# IPL Powerplay Score Prediction with AWS Lambda
![IPL banner](https://cricketaddictor.gumlet.io/wp-content/uploads/2021/02/153757448_173638941016448_6980867142752435675_n.jpg?compress=true&quality=80&w=1920&dpr=2.6)

It's COVID lockdown time. Everyone is worried about the increasing COVID cases, Work From Home (WFH) environment. No way to go out, no parties, no outing, no vacation plans, no gatherings ...... 

Government announces Lockdown : 1.0, 2.0, 3.0, 4.0 .....

At that time every cricket fan had a question. 
> **"Do new Govt. rules in lockdown 4.0 pave the way for IPL 2020 ????"**



Finally IPL 2020 happend. 
* **Even pandamic situation can't derail the IPL juggernaut.**
* IPL 20202 earned revenue of : 4000 crores with 
  *  35% reduced cost and 
  *  25% increase in viewership. 

As a cricket fan I watch all the matches, during that time I observed that **"Powerplay plays a mojor role"** which is very Important in Team score.

## What is powerplay in IPL & it's Importance???
* In summary "powerplay" means fielding restrictions in 1<sup>st</sup> 6-overs.
  * means, only 2-fielders can stay outside the Inner-circle
  * After powerplay, upto 5-fielders can stay outside inner circle & 4-fielders must remain inside the inner circle.
* Which makes the **batting compatively easy.**
* Also **it's a trap for the batsmen**, as this will get them to take into a risk and loose their wickets in 1<sup>st</sup> 6-overs

## Effect of "Powerplay" in IPL matches??
* Powerplay overs are considered as pillars of any teams  victory.
* 75% - of winning chance of the team depends on the Powerplay score.
* So, every team's expectation from the top 3-batsmen is **"START THE INNINGS BIG"**

In this blog post I am going to predict the score of an IPL match at the end of powerplay.

In [1]:
import requests,zipfile,io,json
from pathlib import Path

from sklearn.preprocessing import LabelEncoder
from collections import OrderedDict

import pandas as pd
import seaborn as sns
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
dataset_path = Path(Path.cwd(),'dataset_new')
filename = 'all_matches.csv'

json_path = Path.joinpath(dataset_path, 'label_encode.json')

# 1. Loading the Data Set: 

Dataset will be available at : https://cricsheet.org/downloads/ipl_male_csv.zip

In [3]:
def getCsvFile(url="https://cricsheet.org/downloads/ipl_male_csv2.zip"):
    res = requests.get(url, stream=True)
    if res.status_code == 200:
        print('### Downloading the CSV file')
        z = zipfile.ZipFile(io.BytesIO(res.content))
        if filename in z.namelist():
            z.extract(filename, dataset_path)
            print('### Extracted %s file' % filename)
        else:
            print('### %s : File not found in ZIP Artifact' % filename)

def downloadDataset():
    if not dataset_path.exists():
        Path.mkdir(dataset_path)
        print('### Created Dataset folder')
        getCsvFile()
    elif dataset_path.exists():
        files = [file for file in dataset_path.iterdir() if file.name ==
                 'all_matches.csv']
        if len(files) == 0:
            getCsvFile()
        else:
            print('### File already extracted in given path')

In [4]:
downloadDataset()

### File already extracted in given path


# **2. Data Frames:** 

In [5]:
csv_file=Path.joinpath(dataset_path, filename)

df = pd.read_csv(csv_file,parse_dates=['start_date'],low_memory=False)
df_parsed = df.copy(deep=True)
print(df.shape)
df.head()


(200664, 22)


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **3. Exploratory Data Analysis (EDA) / Data Processing:**
## 3.1: Identify "null" values in Dataset:

In [6]:
df.isnull().sum()

match_id                       0
season                         0
start_date                     0
venue                          0
innings                        0
ball                           0
batting_team                   0
bowling_team                   0
striker                        0
non_striker                    0
bowler                         0
runs_off_bat                   0
extras                         0
wides                     194583
noballs                   199854
byes                      200135
legbyes                   197460
penalty                   200662
wicket_type               190799
player_dismissed          190799
other_wicket_type         200664
other_player_dismissed    200664
dtype: int64

## 3.2: Cleaning Steps:
1. Here columns 'season' and 'start_date'  are not needed for our prediction. ```So we can drop the columns 'season' and 'start_date' from the dataset.```
2. Delete Non-existing teams : 
```
       'Kochi Tuskers Kerala' 'Pune Warriors','Rising Pune Supergiants', 'Rising Pune Supergiant','Gujarat Lions'
```
3. Replace the old team names with new team name:
```
      'Delhi Daredevils'  --> 'Delhi Capitals'
      'Deccan Chargers'   --> 'Sunrisers Hyderabad'
      'Punjab Kings'      --> 'Kings XI Punjab'
```

4. Correct the venue column with unique names. In this dataset same stadium is being represented as in multiple ways. So identify those and rename.
```
      ['M Chinnaswamy Stadium', 'M.Chinnaswamy Stadium']
      ['Brabourne Stadium', 'Brabourne Stadium, Mumbai']
      ['Punjab Cricket Association Stadium, Mohali', 'Punjab Cricket Association IS Bindra Stadium, Mohali', 'Punjab Cricket Association IS Bindra Stadium']
      ['Wankhede Stadium', 'Wankhede Stadium, Mumbai']
      ['Rajiv Gandhi International Stadium, Uppal', 'Rajiv Gandhi International Stadium']
      ['MA Chidambaram Stadium, Chepauk','MA Chidambaram Stadium',      'MA Chidambaram Stadium, Chepauk, Chennai']
```

5. Rename the column names (for easy coding)
```
      'striker'     --> 'batsmen'
      'non-striker' --> 'batsmen_nonstriker'  (This column is not required)
      'bowler'      --> 'bowlers'
```
      
6. Create a columns "Total_score" : which reflects the runs through bat and extra runs through wides,byes,noballs,legbyes...etc.  
Hence we can drop columsn ```['wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type','other_wicket_type', 'other_player_dismissed']```

In [7]:
df_parsed.drop(columns=['season','start_date'],inplace=True)

In [8]:
non_exist_teams = ['Kochi Tuskers Kerala',
                    'Pune Warriors',
                    'Rising Pune Supergiants',
                    'Rising Pune Supergiant',
                    'Gujarat Lions']
mask_bat_team = df_parsed['batting_team'].isin(non_exist_teams)
mask_bow_team = df_parsed['bowling_team'].isin(non_exist_teams)
df_parsed = df_parsed[~mask_bat_team]
df_parsed = df_parsed[~mask_bow_team]

<ipython-input-8-567adafd9830>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_parsed = df_parsed[~mask_bow_team]


In [9]:
df_parsed.loc[df_parsed.batting_team ==
                'Delhi Daredevils', 'batting_team'] = 'Delhi Capitals'
df_parsed.loc[df_parsed.batting_team == 'Deccan Chargers',
                'batting_team'] = 'Sunrisers Hyderabad'
df_parsed.loc[df_parsed.batting_team ==
                'Punjab Kings', 'batting_team'] = 'Kings XI Punjab'

df_parsed.loc[df_parsed.bowling_team ==
                'Delhi Daredevils', 'bowling_team'] = 'Delhi Capitals'
df_parsed.loc[df_parsed.bowling_team == 'Deccan Chargers',
                'bowling_team'] = 'Sunrisers Hyderabad'
df_parsed.loc[df_parsed.bowling_team ==
                'Punjab Kings', 'bowling_team'] = 'Kings XI Punjab'



In [10]:
df_parsed.venue.unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Eden Gardens', 'Wankhede Stadium', 'Sawai Mansingh Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'MA Chidambaram Stadium, Chepauk', 'Dr DY Patil Sports Academy',
       'Newlands', "St George's Park", 'Kingsmead', 'SuperSport Park',
       'Buffalo Park', 'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Brabourne Stadium, Mumbai',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sheikh Zayed Stadium',
       'Sharjah Cricket Stadium', 'Dubai International Cricket Stadium',
      

In [11]:
list(filter(lambda x : 'chidam' in x.lower(),list(df_parsed.venue.unique())))
    

['MA Chidambaram Stadium, Chepauk',
 'MA Chidambaram Stadium',
 'MA Chidambaram Stadium, Chepauk, Chennai']

In [12]:
df_parsed.loc[df_parsed.venue == 'M.Chinnaswamy Stadium',
                'venue'] = 'M Chinnaswamy Stadium'
df_parsed.loc[df_parsed.venue == 'Brabourne Stadium, Mumbai',
                'venue'] = 'Brabourne Stadium'
df_parsed.loc[df_parsed.venue == 'Punjab Cricket Association IS Bindra Stadium, Mohali',
                'venue'] = 'Punjab Cricket Association Stadium'
df_parsed.loc[df_parsed.venue == 'Punjab Cricket Association IS Bindra Stadium',
                'venue'] = 'Punjab Cricket Association Stadium'
df_parsed.loc[df_parsed.venue == 'Wankhede Stadium, Mumbai',
                'venue'] = 'Wankhede Stadium'
df_parsed.loc[df_parsed.venue == 'Rajiv Gandhi International Stadium, Uppal',
                'venue'] = 'Rajiv Gandhi International Stadium'
df_parsed.loc[df_parsed.venue == 'MA Chidambaram Stadium, Chepauk',
                'venue'] = 'MA Chidambaram Stadium'
df_parsed.loc[df_parsed.venue == 'MA Chidambaram Stadium, Chepauk, Chennai',
                'venue'] = 'MA Chidambaram Stadium'

In [13]:
df_parsed = df_parsed.rename(columns={
                                 'striker': 'batsmen', 'non_striker': 'batsmen_non_striker', 'bowler': 'bowlers'})

In [14]:
df_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173645 entries, 0 to 200663
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   match_id                173645 non-null  int64  
 1   venue                   173645 non-null  object 
 2   innings                 173645 non-null  int64  
 3   ball                    173645 non-null  float64
 4   batting_team            173645 non-null  object 
 5   bowling_team            173645 non-null  object 
 6   batsmen                 173645 non-null  object 
 7   batsmen_non_striker     173645 non-null  object 
 8   bowlers                 173645 non-null  object 
 9   runs_off_bat            173645 non-null  int64  
 10  extras                  173645 non-null  int64  
 11  wides                   5316 non-null    float64
 12  noballs                 710 non-null     float64
 13  byes                    468 non-null     float64
 14  legbyes             

In [15]:
df_parsed['Total_score'] = df_parsed.runs_off_bat + df_parsed.extras

df_parsed.drop(columns=['wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type',
                'other_wicket_type', 'other_player_dismissed'], axis=1, inplace=True)

## 3.3: Select required Data:
   Here as we are predicting the IPL match **"Powerplay"** score, we can drop the rest of the balls and run detail through, 
   * by selecting the 1<sup>st</sup> 6-Overs details and  
   * with Innings (1 and 2) : as there is no 3<sup>rd</sup> or 4<sup>th</sup> innings happen in powerplay


In [16]:
df_parsed[(df_parsed.ball < 6.0) & (df_parsed.innings < 3)].to_csv(
    Path.joinpath(dataset_path, '1_parseDataset.csv'), index=False)

print('### Total {} : venue details present '.format(
    len(df_parsed.venue.unique())))
print('### Total {}  : Batting teams are there'.format(
    len(df_parsed.batting_team.unique())))
print('### Total {}  : Bowlling teams are there'.format(
    len(df_parsed.bowling_team.unique())))
print('### Batting teams are : {}'.format(df_parsed.batting_team.unique()))
print('### Bowling teams are : {}'.format(df_parsed.bowling_team.unique()))
print('### Shape of data frame after initial cleanup :{}'.format(df_parsed.shape))


### Total 35 : venue details present 
### Total 8  : Batting teams are there
### Total 8  : Bowlling teams are there
### Batting teams are : ['Kolkata Knight Riders' 'Royal Challengers Bangalore'
 'Chennai Super Kings' 'Kings XI Punjab' 'Rajasthan Royals'
 'Delhi Capitals' 'Sunrisers Hyderabad' 'Mumbai Indians']
### Bowling teams are : ['Royal Challengers Bangalore' 'Kolkata Knight Riders' 'Kings XI Punjab'
 'Chennai Super Kings' 'Delhi Capitals' 'Rajasthan Royals'
 'Sunrisers Hyderabad' 'Mumbai Indians']
### Shape of data frame after initial cleanup :(173645, 13)


In [17]:
df_parsed

,match_id,venue,innings,ball,batting_team,bowling_team,batsmen,batsmen_non_striker,bowlers,runs_off_bat,extras,player_dismissed,Total_score
0,335982,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,1
1,335982,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0
2,335982,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,NaN,1
3,335982,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0
4,335982,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200659,1254086,"Narendra Modi Stadium, Ahmedabad",2,17.2,Delhi Capitals,Kings XI Punjab,SO Hetmyer,S Dhawan,RP Meredith,6,0,NaN,6
200660,1254086,"Narendra Modi Stadium, Ahmedabad",2,17.3,Delhi Capitals,Kings XI Punjab,SO Hetmyer,S Dhawan,RP Meredith,6,0,NaN,6
200661,1254086,"Narendra Modi Stadium, Ahmedabad",2,17.4,Delhi Capitals,Kings XI Punjab,SO Hetmyer,S Dhawan,RP Meredith,4,0,NaN,4
200662,1254086,"Narendra Modi Stadium, Ahmedabad",2,17.5,Delhi Capitals,Kings XI Punjab,SO Hetmyer,S Dhawan,RP Meredith,0,1,NaN,1


In [18]:
df_parsed = pd.read_csv(Path.joinpath(dataset_path, '1_parseDataset.csv'))
df_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54525 entries, 0 to 54524
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   match_id             54525 non-null  int64  
 1   venue                54525 non-null  object 
 2   innings              54525 non-null  int64  
 3   ball                 54525 non-null  float64
 4   batting_team         54525 non-null  object 
 5   bowling_team         54525 non-null  object 
 6   batsmen              54525 non-null  object 
 7   batsmen_non_striker  54525 non-null  object 
 8   bowlers              54525 non-null  object 
 9   runs_off_bat         54525 non-null  int64  
 10  extras               54525 non-null  int64  
 11  player_dismissed     2062 non-null   object 
 12  Total_score          54525 non-null  int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 5.4+ MB


# **4. Encoding:** 

Dataframe column values of Dtype Object:
Till now as a initial step in step-2,
* We cleaned our dataset with all the "null" values and 
* filtered the columns/rows data (which are not used for prediction) and
* Added the required column with values (like total_score) to make the dataset clean.

As you can see above, our cleaned dataset is having 13 columns of multiple Dtype like int64 and float64 and object.

So next what I am going to do is trying to convert all these multiple Dtypes into a single Dtype, to train my model.

## 4.1:  Encode "batsmen" and "bowlers" column values:
   * Here we can see, few players who can bat as well as bowl.
    Means same player will be listed as a batsmen and as well as bowler.
    * So to make the prediction properly, I am creating one Dataframe with all the players name it as "players_df", which I use for encoding the players with some value to identify.
* For inference going further, I create a dictionary with all these encoded values 

Use "sklearn label encoder" module to encode the dataframe

In [19]:
players_df = pd.DataFrame(np.append(
                            df_parsed.batsmen.unique(), df_parsed.bowlers.unique()),
                            columns=['Players']
                        )
players_df

,Players
0,SC Ganguly
1,BB McCullum
2,RT Ponting
3,R Dravid
4,W Jaffer
...,...
605,RP Meredith
606,Abhishek Sharma
607,LI Meriwala
608,Jalaj S Saxena


In [20]:
label_encode_dict = {}

le = LabelEncoder()
le.fit(players_df.Players)
Players_e = le.transform(players_df.Players)
Players_e_inv = le.inverse_transform(Players_e)

label_encode_dict['Players'] = dict(
        zip(Players_e_inv, map(int, Players_e)))

## 4.2: Encode "vennue" and "batting_team" and "bowling_team" column values:

In [21]:
le.fit(df_parsed.venue)
venue_e = le.transform(df_parsed.venue)
venue_e_inv = le.inverse_transform(venue_e)
label_encode_dict['venue'] = dict(zip(venue_e_inv, map(int, venue_e)))

le.fit(df_parsed.batting_team)
batting_team_e = le.transform(df_parsed.batting_team)
batting_team_e_inv = le.inverse_transform(batting_team_e)
label_encode_dict['batting_team'] = dict(
    zip(batting_team_e_inv, map(int, batting_team_e)))

le.fit(df_parsed.bowling_team)
bowling_team_e = le.transform(df_parsed.bowling_team)
bowling_team_e_inv = le.inverse_transform(bowling_team_e)
label_encode_dict['bowling_team'] = dict(
    zip(bowling_team_e_inv, map(int, bowling_team_e)))

## 4.3: Save the encoded values to a json file

In [22]:
with open(json_path, 'w') as f:
    json.dump(label_encode_dict, f)

## 4.4: Format the Dataset:
In this step I am trying to club the all rows with respect to matchID and Innings (as match ID is unique way to identify a particular match and Innings to identify who bat first).

Based on these two details, 
* grab all the batsmen and bowler details who batted and bowled in 1<sup>st</sup> 6-overs
* Calculate the total score (runs through bat + extra runs)
* How many players dismissed in 1st 6-overs


In [23]:
print('### Shape of Dataframe before format_data : {}'.format(df_parsed.shape))
Runs_off_Bat_6_overs = df_parsed.groupby(
    ['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['runs_off_bat'].sum()
Extras_6_overs = df_parsed.groupby(
    ['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['extras'].sum()
TotalScore_6_overs = df_parsed.groupby(
    ['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['Total_score'].sum()

Total_WktsDown = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])[
    'player_dismissed'].count()


### Shape of Dataframe before format_data : (54525, 13)


In [24]:
bat_df = df_parsed.groupby(['match_id', 'venue', 'innings',
                    'batting_team', 'bowling_team'])['batsmen'].apply(list)
bow_df = df_parsed.groupby(['match_id', 'venue', 'innings',
                    'batting_team', 'bowling_team'])['bowlers'].apply(list)

now concat all these formated data into a single dataframe

In [25]:
pd.concat([bat_df, bow_df, Runs_off_Bat_6_overs, Extras_6_overs, TotalScore_6_overs, Total_WktsDown],
          axis=1).to_csv(Path.joinpath(dataset_path,'2_format_data.csv'))

In [26]:
df_parsed = pd.read_csv(Path.joinpath(dataset_path, '2_format_data.csv'))
df_parsed

,match_id,venue,innings,batting_team,bowling_team,batsmen,bowlers,runs_off_bat,extras,Total_score,player_dismissed
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"['SC Ganguly', 'BB McCullum', 'BB McCullum', '...","['P Kumar', 'P Kumar', 'P Kumar', 'P Kumar', '...",51,10,61,1
1,335982,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"['R Dravid', 'W Jaffer', 'W Jaffer', 'W Jaffer...","['AB Dinda', 'AB Dinda', 'AB Dinda', 'AB Dinda...",19,7,26,4
2,335983,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Kings XI Punjab,"['PA Patel', 'PA Patel', 'PA Patel', 'ML Hayde...","['B Lee', 'B Lee', 'B Lee', 'B Lee', 'B Lee', ...",50,3,53,1
3,335983,"Punjab Cricket Association Stadium, Mohali",2,Kings XI Punjab,Chennai Super Kings,"['K Goel', 'K Goel', 'K Goel', 'K Goel', 'JR H...","['JDP Oram', 'JDP Oram', 'JDP Oram', 'JDP Oram...",61,2,63,1
4,335984,Feroz Shah Kotla,1,Rajasthan Royals,Delhi Capitals,"['T Kohli', 'T Kohli', 'T Kohli', 'T Kohli', '...","['GD McGrath', 'GD McGrath', 'GD McGrath', 'GD...",38,2,40,2
...,...,...,...,...,...,...,...,...,...,...,...
1454,1254084,"Arun Jaitley Stadium, Delhi",2,Mumbai Indians,Chennai Super Kings,"['Q de Kock', 'Q de Kock', 'Q de Kock', 'RG Sh...","['DL Chahar', 'DL Chahar', 'DL Chahar', 'DL Ch...",56,2,58,0
1455,1254085,"Arun Jaitley Stadium, Delhi",1,Rajasthan Royals,Sunrisers Hyderabad,"['JC Buttler', 'JC Buttler', 'JC Buttler', 'YB...","['B Kumar', 'B Kumar', 'B Kumar', 'B Kumar', '...",37,5,42,1
1456,1254085,"Arun Jaitley Stadium, Delhi",2,Sunrisers Hyderabad,Rajasthan Royals,"['MK Pandey', 'MK Pandey', 'MK Pandey', 'MK Pa...","['Kartik Tyagi', 'Kartik Tyagi', 'Kartik Tyagi...",55,2,57,0
1457,1254086,"Narendra Modi Stadium, Ahmedabad",1,Kings XI Punjab,Delhi Capitals,"['P Simran Singh', 'P Simran Singh', 'P Simran...","['I Sharma', 'I Sharma', 'I Sharma', 'I Sharma...",39,0,39,2


## 4.5 : Align the batsmen and bowlers details in to a separate column

In above formated dataset, we got list of batsmen and bowlers details who batted and bowled in 6-overs.

Now we have to arrange these batsmen into a separate columns,
```
 * say bat1,bat2,bat3,bat4....bat10
 * say bow1,bow2,bow3.....bow6
```

Here I selected only 10-batsmen (as we have only 10-wickets), and 6-bowlers (can bowl in 6-overs) because in 6-overs this is only possible.

```
For proper prediction the order of batsmen and bowlers given the dataset matters. So we need to keep the order :
* batsmen : who batted 1st,2nd 3rd and 4th ... wicket same
* bowler  : who bowled in 1st,2nd,3rd,4th,5th and 6th overs same
```

## 4.6: Create a batsmen and bowlers dummy dataframe:
Inorder to keep track of the order same, so 1st I am going to create a dummy dataframe 
* with 10-batsmen with column names [bat1,bat2,.....bat9,bat10]
* with 6-bowlers with column names [bow1,bow2,bow3,bow4,bow5,bow6]

In [27]:
df_parsed.shape

(1459, 11)

In [28]:
bat  = pd.DataFrame(np.zeros((df_parsed.shape[0],10),dtype=float),columns=['bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10'])
bowl = pd.DataFrame(np.zeros((df_parsed.shape[0],6),dtype=float),columns=['bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6'])

columns = ['bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10','bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6']
df_bat_bow = pd.concat([bat,bowl],axis=1)
df_bat_bow



,bat1,bat2,bat3,bat4,bat5,bat6,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
pd.concat([df_parsed, df_bat_bow], axis=1).to_csv(Path.joinpath(
    dataset_path, '3_create_batsmen_bowler_df.csv'),                index=False)
df_parsed = pd.read_csv(Path.joinpath(
    dataset_path, '3_create_batsmen_bowler_df.csv'))

In [30]:
df_parsed.head(3)

,match_id,venue,innings,batting_team,bowling_team,batsmen,bowlers,runs_off_bat,extras,Total_score,...,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"['SC Ganguly', 'BB McCullum', 'BB McCullum', '...","['P Kumar', 'P Kumar', 'P Kumar', 'P Kumar', '...",51,10,61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,335982,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"['R Dravid', 'W Jaffer', 'W Jaffer', 'W Jaffer...","['AB Dinda', 'AB Dinda', 'AB Dinda', 'AB Dinda...",19,7,26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,335983,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Kings XI Punjab,"['PA Patel', 'PA Patel', 'PA Patel', 'ML Hayde...","['B Lee', 'B Lee', 'B Lee', 'B Lee', 'B Lee', ...",50,3,53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4.7: Update the batsmen list of elements into each column in the same order:

Means, data which is in list of elements in each matchID and innings into corresponding individual batsmen columns in the same order. 
```
Example, here in below list 1st batsmen is bat1 -> SC Ganguly, 2nd is bat2 -> BB McCullum, 3rd is bat3 -> RT Poting ...etc like 

"['SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'BB McCullum', 'SC Ganguly', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'Mohammad Hafeez', 'Mohammad Hafeez', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'Mohammad Hafeez', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum']"

```


In [31]:
for row,val in enumerate(df_parsed.batsmen):
    tmp = val[1:-1].split(', ')
    tmp_list = list(map(lambda x : x.strip("'"),tmp))
    tmp_list = list(OrderedDict.fromkeys(tmp_list))
    for i,j in enumerate(list(map(lambda x : x.strip("'"),tmp_list))):
        if i==10:
            import pdb;pdb.set_trace()

        col = "bat%i"%(i+1)
        df_parsed[col][row] = j

<ipython-input-31-1c0513554501>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parsed[col][row] = j
C:\Users\skanukollu\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [32]:
for row, val in enumerate(df_parsed.bowlers):
    tmp = val[1:-1].split(', ')
    tmp_list = list(map(lambda x: x.strip("'"), tmp))
    tmp_list = list(OrderedDict.fromkeys(tmp_list))
    for i, j in enumerate(list(map(lambda x: x.strip("'"), tmp_list))):
        col = "bow%i" % (i+1)
        df_parsed[col][row] = j

<ipython-input-32-3e8e2b55314b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parsed[col][row] = j


In [33]:
df_parsed.to_csv(Path.joinpath(dataset_path,
            '4_update_batsmen_bowler_column_names.csv'),                     index=False)
df_parsed = pd.read_csv(Path.joinpath(
    dataset_path, '4_update_batsmen_bowler_column_names.csv'))

In [34]:
df_parsed.head(3)

,match_id,venue,innings,batting_team,bowling_team,batsmen,bowlers,runs_off_bat,extras,Total_score,...,bat7,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6
0,335982,M Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,"['SC Ganguly', 'BB McCullum', 'BB McCullum', '...","['P Kumar', 'P Kumar', 'P Kumar', 'P Kumar', '...",51,10,61,...,0.0,0.0,0.0,0.0,P Kumar,Z Khan,AA Noffke,0.0,0.0,0.0
1,335982,M Chinnaswamy Stadium,2,Royal Challengers Bangalore,Kolkata Knight Riders,"['R Dravid', 'W Jaffer', 'W Jaffer', 'W Jaffer...","['AB Dinda', 'AB Dinda', 'AB Dinda', 'AB Dinda...",19,7,26,...,0.0,0.0,0.0,0.0,AB Dinda,I Sharma,AB Agarkar,0.0,0.0,0.0
2,335983,"Punjab Cricket Association Stadium, Mohali",1,Chennai Super Kings,Kings XI Punjab,"['PA Patel', 'PA Patel', 'PA Patel', 'ML Hayde...","['B Lee', 'B Lee', 'B Lee', 'B Lee', 'B Lee', ...",50,3,53,...,0.0,0.0,0.0,0.0,B Lee,S Sreesanth,JR Hopes,0.0,0.0,0.0


So finally our dataframe is ready with batsmen and bowlers details.

So we can drop few columns which are not important.

In [35]:
df_model = df_parsed[['venue', 'innings', 'batting_team', 'bowling_team', 'bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8','bat9', 'bat10', 'bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6', 'runs_off_bat', 'extras', 'Total_score', 'player_dismissed']]
df_model.columns

Index(['venue', 'innings', 'batting_team', 'bowling_team', 'bat1', 'bat2',
       'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10', 'bow1',
       'bow2', 'bow3', 'bow4', 'bow5', 'bow6', 'runs_off_bat', 'extras',
       'Total_score', 'player_dismissed'],
      dtype='object')

In [36]:
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_csv(sep="|", index=False)))

pandas_df_to_markdown_table(df_model.head())

venue|innings|batting_team|bowling_team|bat1|bat2|bat3|bat4|bat5|bat6|bat7|bat8|bat9|bat10|bow1|bow2|bow3|bow4|bow5|bow6|runs_off_bat|extras|Total_score|player_dismissed
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---
M Chinnaswamy Stadium|1|Kolkata Knight Riders|Royal Challengers Bangalore|SC Ganguly|BB McCullum|RT Ponting|0.0|0.0|0.0|0.0|0.0|0.0|0.0|P Kumar|Z Khan|AA Noffke|0.0|0.0|0.0|51|10|61|1
M Chinnaswamy Stadium|2|Royal Challengers Bangalore|Kolkata Knight Riders|R Dravid|W Jaffer|V Kohli|JH Kallis|CL White|MV Boucher|0.0|0.0|0.0|0.0|AB Dinda|I Sharma|AB Agarkar|0.0|0.0|0.0|19|7|26|4
Punjab Cricket Association Stadium, Mohali|1|Chennai Super Kings|Kings XI Punjab|PA Patel|ML Hayden|MEK Hussey|0.0|0.0|0.0|0.0|0.0|0.0|0.0|B Lee|S Sreesanth|JR Hopes|0.0|0.0|0.0|50|3|53|1
Punjab Cricket Association Stadium, Mohali|2|Kings XI Punjab|Chennai Super Kings|K Goel|JR Hopes|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|JDP Oram|MS Gony|0.0|0.0|0.0|0.0|61|2|63|1
Feroz Shah Kotla|1|Rajasthan Royals|Delhi Capitals|T Kohli|YK Pathan|SR Watson|M Kaif|0.0|0.0|0.0|0.0|0.0|0.0|GD McGrath|B Geeves|MF Maharoof|0.0|0.0|0.0|38|2|40|2


# 4.8: Encode the multiple Dtypes into single Dtype:

Now its time to use, the label encoded values (already done in previous steps) to encode the dataframe.

In [37]:
json_path = Path.joinpath(dataset_path, 'label_encode.json')
with open(json_path) as f:
    data = json.load(f)

condition = False

for col in df_model.columns:
    if col in data.keys():
        condition = True
        col = col
    elif col in ['bat1', 'bat2','bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10']:
        condition = True
        col = 'Players' #'batsmen'
    elif col in ['bow1','bow2', 'bow3', 'bow4', 'bow5', 'bow6']:
        col = 'Players' #'bowlers'
        condition = True

    if condition:
        condition = False
        for key in data[col]:
            df_model = df_model.replace([key], data[col][key])

df_model.to_csv(Path.joinpath(dataset_path, '5_model_df.csv'), index=False)

So finally dataset is ready. 
Next step is prepare model for training 

In [38]:
pandas_df_to_markdown_table(df_model.head())

venue|innings|batting_team|bowling_team|bat1|bat2|bat3|bat4|bat5|bat6|bat7|bat8|bat9|bat10|bow1|bow2|bow3|bow4|bow5|bow6|runs_off_bat|extras|Total_score|player_dismissed
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---
15|1|3|6|419|70|385|0.0|0.0|0.0|0.0|0.0|0.0|0.0|326|512|19|0.0|0.0|0.0|51|10|61|1
15|2|6|3|351|496|483|187|97|297|0.0|0.0|0.0|0.0|22|163|20|0.0|0.0|0.0|19|7|26|4
23|1|0|2|331|286|274|0.0|0.0|0.0|0.0|0.0|0.0|0.0|65|408|201|0.0|0.0|0.0|50|3|53|1
23|2|2|0|207|201|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|184|296|0.0|0.0|0.0|0.0|61|2|63|1
10|1|5|1|468|506|443|255|0.0|0.0|0.0|0.0|0.0|0.0|146|62|275|0.0|0.0|0.0|38|2|40|2


## 5 Normalization: